
## High Court dataset: cleaning, feature engineering, insights, and modeling
- Predict monthly disposals (regression)
- Classify backlog risk (classification)


## Importing all the neccessary libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Mounting my google drive to load my dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

## Loading our dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/PROJECTS/AIDTM/High Court-Pro5(in).csv")

In [ ]:
df.head(5)

## Analyzing my dataset

In [ ]:
df.info()

# There are no null values in my dataset.

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
# Renaming the features

rename_map = {
    "Cases That Were Instituted In Last Month (UOM:Number), Scaling Factor:1": "instituted_last_month",
    "Cases That Were Disposed In Last Month (UOM:Number), Scaling Factor:1": "disposed_last_month",
    "Pending Cases Of Age Less Than One Year (UOM:Number), Scaling Factor:1": "pending_lt1y",
    "Cases Pending For A Period Of 1 To 3 Years At High Court (UOM:Number), Scaling Factor:1": "pending_1to3y",
    "Cases Pending For A Period Of 3 To 5 Years At High Court (UOM:Number), Scaling Factor:1": "pending_3to5y",
    "Cases Pending For A Period Of 5 To 10 Years At High Court (UOM:Number), Scaling Factor:1": "pending_5to10y",
    "Pending Cases Of Age Above 10 Years (UOM:Number), Scaling Factor:1": "pending_gt10y",
    "Writ Petition Cases (UOM:Number), Scaling Factor:1": "writ_cases",
    "Second Appeal Cases (UOM:Number), Scaling Factor:1": "second_appeal_cases",
    "First Appeal Cases (UOM:Number), Scaling Factor:1": "first_appeal_cases",
    "Appeal Cases (UOM:Number), Scaling Factor:1": "appeal_cases",
    "Petition Cases (UOM:Number), Scaling Factor:1": "petition_cases",
    "Revision Cases (UOM:Number), Scaling Factor:1": "revision_cases",
    "Reference Cases (UOM:Number), Scaling Factor:1": "reference_cases",
    "Suit Cases (UOM:Number), Scaling Factor:1": "suit_cases",
    "Review Cases (UOM:Number), Scaling Factor:1": "review_cases",
    "Application Cases (UOM:Number), Scaling Factor:1": "application_cases",
    "Cases That Were Instituted In Current Year (UOM:Number), Scaling Factor:1": "instituted_current_year",
    "Cases That Were Disposed In Current Year (UOM:Number), Scaling Factor:1": "disposed_current_year",
    "Country": "country",
    "Year": "year_str",
    "Calendar Day": "calendar_day",
    "High Court Name": "high_court_name",
    "High Court Bench": "high_court_bench",
    "Court Case Type": "court_case_type",
}
df.rename(columns=rename_map, inplace=True)

In [ ]:
# To check the data type and null values.
df.info()

In [ ]:
df.head(3)

In [ ]:
# Droping the year column as our dataset is of only one year i.e., 2025 and it is already mentioned on Calender_day column. Also, country column is not giving any significant output from here

df.drop(columns=['year_str', 'country'], inplace=True)

In [ ]:
df.head(3)

In [ ]:
# Changing the data type of Calender_day to Date format

df['calendar_day'] = pd.to_datetime(df['calendar_day'], format="%m/%d/%Y")

In [ ]:
df.head(3)

In [ ]:
df.info()

### Q2 Which Court Case Type has the highest total pending cases (all age groups combined)?

In [ ]:
# Making a new column which have the sum of all pending cases for Civil and Criminal

df['total_pending_cases'] = (
    df['pending_lt1y'] +
    df['pending_1to3y'] +
    df['pending_3to5y'] +
    df['pending_5to10y'] +
    df['pending_gt10y']
)

case_type_totals = df.groupby('court_case_type')['total_pending_cases'].sum()
highest_value = case_type_totals.max()
highest_case_type = case_type_totals.idxmax()

print(f"{highest_case_type} has the highest total pending cases: {highest_value}")



In [ ]:
case_type_totals

### Q3 What is the median number of writ petition cases across all High Courts?

In [ ]:
writ_petition_cases = df['writ_cases'].median()
print(f"The median number of writ petition cases across all High Courts is: {int(writ_petition_cases)}")

### Q4 Which High Court has the maximum number of total appeal cases?

In [ ]:
total_appeal_cases = df.groupby('high_court_name')['appeal_cases'].sum()
max_appeal_cases = total_appeal_cases.max()
max_appeal_court = total_appeal_cases.idxmax()

print(f"{max_appeal_court} has the maximum number of total appeal cases: {int(max_appeal_cases)}")

### Q5 Add a new column BacklogRatio = Pending Cases Above 10 Years ÷ Total Pending Cases.

In [ ]:
df['BacklogRatio'] = df['pending_gt10y'] / df['total_pending_cases']
df['BacklogRatio'] = df['BacklogRatio'].round(3)

In [ ]:
df['BacklogRatio'].head(2)

### Q6 Show the top 5 High Courts with the highest BacklogRatio.

In [ ]:
top_5_high_courts = df.sort_values(by='BacklogRatio', ascending=False).head(5)
top_5_high_courts[['high_court_name', 'BacklogRatio']]

### Q7 Which High Court has the maximum application cases?

In [ ]:
max_cases = df.sort_values(by='application_cases', ascending=False).head(1)
print(f"{max_cases['high_court_name'].values[0]} has the maximum application cases with {max_cases['application_cases'].values[0]} cases.")


### Q8 Find the average number of disposed cases in the current year grouped by Court Case Type.

In [ ]:
disposed_cases = df.groupby('court_case_type')['disposed_current_year'].mean()
print(disposed_cases)

### Q9 Get summary statistics (mean, min, max) of instituted cases last month.

In [ ]:
summary = df['instituted_last_month'].agg(['mean', 'min', 'max'])
print((summary).round(2))

### Q10 From the DataFrame, select the first 10 High Courts where pending cases above 10 years form more than 20% of total pending cases.

In [ ]:
epending_cases_above_10 = df[df['pending_gt10y'] > 0.2 * df['total_pending_cases']]
epending_cases_above_10 = epending_cases_above_10.head(10)
epending_cases_above_10[['high_court_name','total_pending_cases']]

### Q11 Create a Series of total instituted cases (last month) by High Court using indexing and display the top 5 with the highest values.

In [ ]:
total_instituted_cases = df.sort_values(by= 'instituted_last_month', ascending = False).head(5)
total_instituted_cases[['high_court_name', 'instituted_last_month']]

In [ ]:
df.info()

In [ ]:
df.drop(columns = 'BacklogRatio', inplace = True)

# My Insights

In [ ]:
df.shape

In [ ]:
df.head(3)

#### Legal insight:

Net flow indicates whether the docket is accumulating or clearing. Persistent positive net flow signals rising backlog, requiring capacity or process interventions (listing discipline, additional benches, ADR).

Clearance rate (YTD) = the share of cases disposed relative to cases instituted in the current year so far
If clearance_rate_ytd = 1 (100%) → the court disposed as many cases as were filed this year → backlog didn’t grow.

If < 1 → more cases filed than disposed → backlog is increasing.

If > 1 → court is disposing more cases than filed this year (means they are also clearing old backlog).

In [ ]:
# Net flow (monthly backlog movement): positive = backlog grew; negative = backlog reduced
df["net_flow_last_month"] = df["instituted_last_month"] - df["disposed_last_month"]
x = df[["net_flow_last_month", "high_court_name"]].sort_values(
    by="net_flow_last_month",
    ascending=False
).head(5)
x

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(
    data=x,
    x="net_flow_last_month",
    y="high_court_name",
    palette="Reds_r"
)
plt.title("Top 3 High Courts by Backlog Growth (Net Flow Last Month)")

#### These are the Top 5 courts where there are way much number of backlogs.

In [ ]:
df.info()

In [ ]:
# Net flow (monthly backlog movement): positive = backlog grew; negative = backlog reduced
df["net_flow_current_year"] = df["instituted_current_year"] - df["disposed_current_year"]
y = df[["net_flow_current_year", "high_court_name"]].sort_values(
    by="net_flow_current_year",
    ascending=False
).head(5)
y

#### As the no. of backlogs of monthly and yearly are different, because there might be a case whiere in few months the backlogs got increases by maimum numbers hence the overall yearly value go increases.

So, this didn't give proper info.

In [ ]:
df.info()

In [ ]:
# Aggregate backlog movement by court-bench
backlog_bench = df.groupby(
    ["high_court_name", "high_court_bench"]
)["net_flow_last_month"].sum().reset_index()

# Top 5 backlog growth (instituted > disposed) and reduction (disposed > instituted)
top_growth = backlog_bench.nlargest(5, "net_flow_last_month")
top_reduction = backlog_bench.nsmallest(5, "net_flow_last_month")

print("\nTop 5 backlog growth courts/benches:")
print(top_growth[["high_court_name", "high_court_bench", "net_flow_last_month"]])

print("\nTop 5 backlog reduction courts/benches:")
print(top_reduction[["high_court_name", "high_court_bench", "net_flow_last_month"]])


### Insights from this analysis

#### Top backlog growth benches
- These are benches where instituted cases > disposed cases last month.
- High positive net flow → backlog is increasing quickly.
- Signals resource constraints, rising inflow, or inefficiency at those benches.

#### Top backlog reduction benches
- These benches disposed more cases than instituted last month.
- Negative net flow → backlog is reducing.
- Shows strong performance — courts are not only keeping up with new cases but also clearing pending ones.

In [ ]:
# Backlog Ratio = total pending cases / disposed cases (last month)
df["BacklogRatio"] = df["total_pending_cases"] / df["disposed_last_month"].replace(0, np.nan)


In [ ]:
df["clearance_rate_ytd"] = (
    df["disposed_current_year"] / df["instituted_current_year"].replace(0, np.nan)
).fillna(0).round(2)

In [ ]:
# Define a simple risk flag (customizable)
df["risk_flag"] = (
    (df["clearance_rate_ytd"] < 0.9) | (df["BacklogRatio"] > 1.2)
).astype(int)

# Courts with most high-risk benches
risk_counts = df.groupby("high_court_name")["risk_flag"].sum().nlargest(5)
print("\nCourts with most high-risk benches:")
print(risk_counts)


In [ ]:
# Lowest clearance rate courts (avg YTD)
clearance_by_court = df.groupby("high_court_name")["clearance_rate_ytd"].mean().nsmallest(5)

# Courts with most high-risk benches
risk_counts = df.groupby("high_court_name")["risk_flag"].sum().nlargest(5)

# Top 5 benches by BacklogRatio
top5_backlogratio = (
    df.dropna(subset=["BacklogRatio"])
      .nlargest(5, "BacklogRatio")
      [["high_court_name","high_court_bench","court_case_type","BacklogRatio"]]
)

print("\nLowest clearance rate courts (avg YTD):")
print(clearance_by_court)

print("\nCourts with most high-risk benches:")
print(risk_counts)

print("\nTop 5 benches by BacklogRatio:")
print(top5_backlogratio)


#### Lowest Clearance Rate Courts (avg YTD)
- Shows which courts struggle most to dispose of cases relative to new filings this year.
- Low clearance rate = high long-term pendency risk.
- These courts need capacity improvements (more judges, faster procedures, tech adoption).

####Courts with Most High-Risk Benches
- Risk flags indicate benches with structural problems (low clearance, high backlog, low disposal trends, etc.).
- Courts with many flagged benches = systemic risk → not just one inefficient bench, but a widespread issue across that High Court.

#### Top 5 Benches by BacklogRatio
- BacklogRatio is your custom “pressure” metric (pending vs disposed balance).
- Benches at the top are most overloaded → likely causing delays and affecting litigants most severely.
- These can be targeted for urgent intervention (temporary judges, case redistribution).

In [ ]:
df.info()

In [ ]:
# Create heavy_aging_share if heavy_pending_cases exists
if "heavy_pending_cases" in df.columns and "total_pending_cases" in df.columns:
    df["heavy_aging_share"] = df["heavy_pending_cases"] / df["total_pending_cases"]


In [ ]:
df["heavy_aging_share"] = df["pending_gt10y"] / df["total_pending_cases"]

In [ ]:
# Top 10 benches with longest months-to-clear
long_horizon = df.nlargest(10, "months_to_clear")[[
    "high_court_name","high_court_bench","court_case_type","months_to_clear","total_pending_cases"
]]

print("\nBenches with longest months-to-clear:")
print(long_horizon.to_string(index=False))

# Top 10 benches with highest heavy aging share (only if column exists)
if "heavy_aging_share" in df.columns:
    heavy_aging = df.nlargest(10, "heavy_aging_share")[[
        "high_court_name","high_court_bench","court_case_type","heavy_aging_share","total_pending_cases"
    ]]
    print("\nBenches with highest heavy aging share:")
    print(heavy_aging.to_string(index=False))
else:
    print("\n⚠️ Column 'heavy_aging_share' not found. You may need to calculate it first.")

In [ ]:
# Make sure total pending is defined
df["age_total"] = (
    df["pending_lt1y"] + df["pending_1to3y"] + df["pending_3to5y"] +
    df["pending_5to10y"] + df["pending_gt10y"]
)

# Avoid divide-by-zero
df["age_total"] = df["age_total"].replace(0, np.nan)

# Create share columns
df["share_lt1y"]   = (df["pending_lt1y"]   / df["age_total"]).fillna(0)
df["share_1to3y"]  = (df["pending_1to3y"]  / df["age_total"]).fillna(0)
df["share_3to5y"]  = (df["pending_3to5y"]  / df["age_total"]).fillna(0)
df["share_5to10y"] = (df["pending_5to10y"] / df["age_total"]).fillna(0)
df["share_gt10y"]  = (df["pending_gt10y"]  / df["age_total"]).fillna(0)


In [ ]:
num_features = [
    "instituted_last_month", "total_pending_cases", "net_flow_last_month",
    "clearance_rate_ytd",
    "share_lt1y", "share_1to3y", "share_3to5y", "share_5to10y", "share_gt10y",
    "BacklogRatio"
]
cat_features = ["high_court_name", "high_court_bench", "court_case_type"]
target = "disposed_last_month"

In [ ]:
X = df[num_features + cat_features].copy()
y = df[target].values

In [ ]:
# Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [ ]:
preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),  # fill numeric NaNs
        ("scaler", StandardScaler())
    ]), num_features),

    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),  # fill categorical NaNs
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]), cat_features)
])

In [ ]:
# Train model
# -----------------------------
reg_model.fit(X_train, y_train)

In [ ]:
# Evaluate on test set
# -----------------------------
y_pred = reg_model.predict(X_test)

In [ ]:
print("\n=== Regression performance (monthly disposals) ===")
print(f"R²: {r2_score(y_test, y_pred):.3f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):,.0f} cases")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):,.0f} cases")

In [ ]:
# Apply model to all rows (no NaNs dropped)
# -----------------------------
df["pred_disposed_last_month"] = reg_model.predict(X)
df["disposal_gap"] = df["pred_disposed_last_month"] - df["disposed_last_month"]

In [ ]:
# Top 10 underperformers (actual < predicted)
# -----------------------------
underperformers = df.nsmallest(10, "disposal_gap")[
    ["high_court_name","high_court_bench","court_case_type",
     "disposed_last_month","pred_disposed_last_month","disposal_gap"]
]

print("\nLargest negative disposal gaps (actual < expected):")
print(underperformers.to_string(index=False))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
sns.barplot(
    data=underperformers,
    y="high_court_bench",
    x="disposal_gap",
    hue="court_case_type",
    dodge=False,
    palette="Reds_r"
)
plt.title("Top 10 Underperforming Benches (Actual < Predicted)")
plt.xlabel("Disposal Gap (cases)")
plt.ylabel("Bench")
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score

df["risk_flag"] = ((df["clearance_rate_ytd"] < 0.9) | (df["BacklogRatio_struct"] > 0.2)).astype(int)

Xc = df[num_features + cat_features]
yc = df["risk_flag"]

clf_model = Pipeline([
    ("pre", pre),
    ("rf", RandomForestClassifier(n_estimators=400, class_weight="balanced", random_state=42))
])

roc, f1s = [], []
for tr, te in tscv.split(Xc):
    clf_model.fit(Xc.iloc[tr], yc.iloc[tr])
    proba = clf_model.predict_proba(Xc.iloc[te])[:,1]
    preds = (proba >= 0.5).astype(int)
    roc.append(roc_auc_score(yc.iloc[te], proba))
    f1s.append(f1_score(yc.iloc[te], preds))

print(f"Classification CV ROC-AUC: {np.mean(roc):.3f}, F1: {np.mean(f1s):.3f}")


In [ ]:
import pyodbc
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1️⃣ Connect to SQL Server and load data
# ---------------------------------------------------------
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=.\\SQLEXPRESS01;"
    "Database=HighCourtDB1;"
    "Trusted_Connection=yes;"
)

conn = pyodbc.connect(conn_str)
query = "SELECT * FROM HighCourtCases"
df = pd.read_sql(query, conn)
conn.close()

print("\n✅ Data Loaded Successfully — Sample:")
print(df.head())

# ---------------------------------------------------------
# 2️⃣ Pending Cases Summary (Graph: “Numbers That Demand Attention”)
# ---------------------------------------------------------
pending_summary = (
    df.groupby("HighCourtName")["PendingCases"]
    .sum()
    .sort_values(ascending=False)
)
print("\n📊 Top High Courts by Pending Cases:")
print(pending_summary.head(10))

# ---------------------------------------------------------
# 3️⃣ Top 5 Benches — Civil & Criminal Cases (Graph: “Top 5 Benches”)
# ---------------------------------------------------------
if "CivilCasesFiled" in df.columns and "CriminalCasesFiled" in df.columns:
    top_civil = df.sort_values("CivilCasesFiled", ascending=False).head(5)
    top_criminal = df.sort_values("CriminalCasesFiled", ascending=False).head(5)
    print("\n🏛️ Top 5 Benches — Civil Cases Filed (2025):")
    print(top_civil[["HighCourtName", "HighCourtBench", "CivilCasesFiled"]])
    print("\n⚖️ Top 5 Benches — Criminal Cases Filed (2025):")
    print(top_criminal[["HighCourtName", "HighCourtBench", "CriminalCasesFiled"]])
else:
    print("\n⚠️ Columns for CivilCasesFiled or CriminalCasesFiled not found. Skipping top bench analysis.")

# ---------------------------------------------------------
# 4️⃣ Case Disposal Rates (Graph: “Case Disposal Rates: Are Cases Being Solved?”)
# ---------------------------------------------------------
if all(c in df.columns for c in ["CivilDisposed", "CivilInstituted", "CriminalDisposed", "CriminalInstituted"]):
    df["Civil_Disposal_Rate"] = np.where(df["CivilInstituted"] > 0,
                                         df["CivilDisposed"] / df["CivilInstituted"], np.nan)
    df["Criminal_Disposal_Rate"] = np.where(df["CriminalInstituted"] > 0,
                                            df["CriminalDisposed"] / df["CriminalInstituted"], np.nan)
    print("\n📈 Civil Disposal Rate — Average:", df["Civil_Disposal_Rate"].mean().round(2))
    print("📈 Criminal Disposal Rate — Average:", df["Criminal_Disposal_Rate"].mean().round(2))
else:
    print("\n⚠️ Disposal columns not found. Skipping disposal rate analysis.")

# ---------------------------------------------------------
# 5️⃣ Civil & Criminal Case Type Distribution (Graph: “Civil/Criminal Cases by Type”)
# ---------------------------------------------------------
if "CaseCategory" in df.columns and "CaseType" in df.columns and "CasesFiled" in df.columns:
    civil_cases = df[df["CaseCategory"].str.lower() == "civil"].groupby("CaseType")["CasesFiled"].sum()
    criminal_cases = df[df["CaseCategory"].str.lower() == "criminal"].groupby("CaseType")["CasesFiled"].sum()
    print("\n📘 Civil Cases by Type:")
    print(civil_cases.sort_values(ascending=False))
    print("\n📕 Criminal Cases by Type:")
    print(criminal_cases.sort_values(ascending=False))
else:
    print("\n⚠️ CaseCategory/CaseType columns not found. Skipping case-type distribution.")

# ---------------------------------------------------------
# 6️⃣ Benches with High Writ Petition Focus (Graph: “High Focus on Writ Petitions”)
# ---------------------------------------------------------
if all(c in df.columns for c in ["WritPetitions", "TotalCases"]):
    df["Writ_Petition_Percent"] = np.where(df["TotalCases"] > 0,
                                           df["WritPetitions"] / df["TotalCases"] * 100, 0)
    high_writ_benches = df[df["Writ_Petition_Percent"] > 80]
    print("\n📜 Benches with >80% Writ Petitions:")
    print(high_writ_benches[["HighCourtName", "HighCourtBench", "Writ_Petition_Percent"]])
else:
    print("\n⚠️ Columns for WritPetitions or TotalCases missing. Skipping Writ analysis.")

# ---------------------------------------------------------
# 7️⃣ Appeal-Dominant Courts (Graph: “Appeal-Dominant Courts”)
# ---------------------------------------------------------
if all(c in df.columns for c in ["AppealCases", "TotalCases"]):
    df["Appeal_Ratio"] = np.where(df["TotalCases"] > 0,
                                  df["AppealCases"] / df["TotalCases"] * 100, 0)
    appeal_dominant = df[df["Appeal_Ratio"] > 70]
    print("\n⚖️ Appeal-Dominant Benches (>70% Appeals):")
    print(appeal_dominant[["HighCourtName", "HighCourtBench", "Appeal_Ratio"]])
else:
    print("\n⚠️ Columns for AppealCases or TotalCases missing. Skipping appeal dominance analysis.")

# ---------------------------------------------------------
# 8️⃣ Revision vs Appeal Comparison (Graph: “Revision Cases Outnumbering Appeals”)
# ---------------------------------------------------------
if all(c in df.columns for c in ["RevisionCases", "AppealCases"]):
    revision_vs_appeal = df[df["RevisionCases"] > df["AppealCases"]]
    print("\n🔍 Benches where Revision Cases > Appeal Cases:")
    print(revision_vs_appeal[["HighCourtName", "HighCourtBench", "RevisionCases", "AppealCases"]])
else:
    print("\n⚠️ Columns for RevisionCases or AppealCases missing. Skipping revision vs appeal analysis.")

# ---------------------------------------------------------
# 9️⃣ Optional: Summary Stats
# ---------------------------------------------------------
print("\n📊 Summary Statistics for Key Columns:")
print(df.describe(include='all'))
